In [1]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 1.7MB/s 


In [6]:
!pip install xgboost

In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_carprice/'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_carprice


In [5]:
ls

data/                      day_3_simple_model.ipynb  LICENSE
day_2_visualization.ipynb  day_4_moremodels.ipynb    README.md


In [7]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [8]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = df[feat].factorize()[0]

In [10]:
cat_feats = [ x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [ x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values
  model = model
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## Decision Tree

In [15]:
run_model(DecisionTreeRegressor(max_depth=5, random_state=2020), cat_feats)

(-19651.80834171165, 281.0725150830279)

## Random Forest

In [18]:
run_model(RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_split=25, random_state=2020), cat_feats)

(-18815.169847226032, 209.06363146879656)

## XGBoost

In [20]:
xgbparams = {
    'n_estimators': 50, 
    'max_depth': 5,
    'learning_rate': 0.3,
    'seed': 0,
    'random_state': 2020
}

model = xgb.XGBRegressor(**xgbparams)
run_model(model, cat_feats)

[16:57:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:58:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:58:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:59:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:59:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-12427.388965820912, 1628.7143051531575)

In [21]:
m = xgb.XGBRegressor(**xgbparams)
m.fit(X,  y)

importances = PermutationImportance(m, random_state=2020).fit(X, y)
eli5.show_weights(importances, feature_names=cat_feats)

[17:00:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1603 ± 0.0026,param_rok-produkcji__cat
0.1037 ± 0.0021,param_stan__cat
0.0824 ± 0.0014,param_napęd__cat
0.0549 ± 0.0021,param_marka-pojazdu__cat
0.0478 ± 0.0011,param_skrzynia-biegów__cat
0.0455 ± 0.0019,param_moc__cat
0.0403 ± 0.0013,param_faktura-vat__cat
0.0207 ± 0.0008,param_typ__cat
0.0198 ± 0.0009,param_wersja__cat
0.0185 ± 0.0005,seller_name__cat


In [0]:
feats = ['param_rok-produkcji__cat','param_stan__cat','param_napęd__cat','param_marka-pojazdu__cat','param_skrzynia-biegów__cat','param_moc__cat','param_faktura-vat__cat','param_typ__cat','param_wersja__cat','seller_name__cat','param_pojemność-skokowa__cat','param_model-pojazdu__cat','param_kod-silnika__cat','feature_kamera-cofania__cat','feature_wspomaganie-kierownicy__cat','param_uszkodzony__cat','feature_światła-led__cat','param_liczba-miejsc__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']

In [23]:
len(feats)

20

In [24]:
model = xgb.XGBRegressor(**xgbparams)
run_model(model, feats)

[17:10:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:10:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:10:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:10:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:10:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11364.290196435384, 75.38136429951695)

In [27]:
n = 'NOn'
str(n).lower()

'non'

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x).lower()=='none' else int(x))

In [30]:
feats = ['param_rok-produkcji','param_stan__cat','param_napęd__cat','param_marka-pojazdu__cat','param_skrzynia-biegów__cat','param_moc__cat','param_faktura-vat__cat','param_typ__cat','param_wersja__cat','seller_name__cat','param_pojemność-skokowa__cat','param_model-pojazdu__cat','param_kod-silnika__cat','feature_kamera-cofania__cat','feature_wspomaganie-kierownicy__cat','param_uszkodzony__cat','feature_światła-led__cat','param_liczba-miejsc__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']
model = xgb.XGBRegressor(**xgbparams)
run_model(model, feats)

[17:19:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:19:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:19:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:19:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:19:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9834.890639751144, 84.9256187433544)

In [41]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x).lower()=='none' else int(x.split(' ')[0]))

AttributeError: ignored

In [43]:
df['param_moc'].unique()

array([ 90, 115, 262, 110, 310, 105, 140, 175, 125, 185, 190, 440, 141,
       200, 224,  75,  99, 184, 109, 233, 116,  68, 286, 126, 160, 135,
       120, 272,  -1, 150, 180, 136, 102, 131, 218, 245, 170, 112, 250,
       252,  73, 100, 313, 101, 285,  70, 383, 174, 277, 132, 130, 215,
        60, 330, 163, 177,  98,  78, 189, 156, 143,  69, 113,  65, 122,
        82, 251,  95, 197, 235, 238, 171, 381, 400, 178,  80, 165,  85,
       258, 142, 204, 124,  55, 144, 231, 248, 152, 181, 210, 340, 129,
       147,  50,  54, 290, 306, 193,  77, 164,  96, 194, 111, 166, 206,
       118, 360, 211, 271, 455, 280, 106, 114, 421,  74, 213, 121, 275,
       435, 384, 326,  88, 220, 260,  64,  86, 128, 256, 240, 244, 162,
       237, 350,  35, 265, 202, 133,  83, 117, 146,  92, 192, 145, 525,
       254, 182, 328, 367, 148, 456,  97, 270, 107, 108, 203, 155,  94,
        93, 241,  20,  71, 173,  58, 205, 236,   1, 557,  84, 457,  72,
       295, 134, 425, 228,  81, 230, 201,  87, 234, 299, 585, 20

In [44]:
feats = ['param_rok-produkcji','param_stan__cat','param_napęd__cat','param_marka-pojazdu__cat','param_skrzynia-biegów__cat','param_moc','param_faktura-vat__cat','param_typ__cat','param_wersja__cat','seller_name__cat','param_pojemność-skokowa__cat','param_model-pojazdu__cat','param_kod-silnika__cat','feature_kamera-cofania__cat','feature_wspomaganie-kierownicy__cat','param_uszkodzony__cat','feature_światła-led__cat','param_liczba-miejsc__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']
model = xgb.XGBRegressor(**xgbparams)
run_model(model, feats)

[17:25:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:25:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:25:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:25:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:26:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-8647.517299762827, 79.77038592672557)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x).lower()=='none' else int(x.split('cm')[0].replace(' ', '')))

In [46]:
feats = ['param_rok-produkcji','param_stan__cat','param_napęd__cat','param_marka-pojazdu__cat','param_skrzynia-biegów__cat','param_moc','param_faktura-vat__cat','param_typ__cat','param_wersja__cat','seller_name__cat','param_pojemność-skokowa','param_model-pojazdu__cat','param_kod-silnika__cat','feature_kamera-cofania__cat','feature_wspomaganie-kierownicy__cat','param_uszkodzony__cat','feature_światła-led__cat','param_liczba-miejsc__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat']
model = xgb.XGBRegressor(**xgbparams)
run_model(model, feats)

[17:29:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:29:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:29:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:29:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:29:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-8573.002573351745, 100.36364939678465)